In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/bcr_full_v3.csv", header=1)
print(df.shape)
df.head()

(8604, 70)


/tmp/ipykernel_166217/2915050121.py:1: DtypeWarning: Columns (14,18,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/bcr_full_v3.csv", header=1)


,Group IRI,IEDB Receptor ID,Reference Name,Type,IEDB IRI,IEDB IRI.1,Name,Source Molecule,Source Organism,Type.1,...,CDR1 Start Curated.1,CDR1 End Curated.1,CDR1 Start Calculated.1,CDR1 End Calculated.1,CDR2 Curated.1,CDR2 Calculated.1,CDR2 Start Curated.1,CDR2 End Curated.1,CDR2 Start Calculated.1,CDR2 End Calculated.1
0,http://www.iedb.org/receptor/1,1,26-2F,heavylight,http://www.iedb.org/reference/279,http://www.iedb.org/epitope/2134780,"G58, L59, S61, P62, C63, K64, D65, G109, G110,...",Angiogenin,Homo sapiens (human),B cell,...,NaN,NaN,27.0,36.0,NaN,AAS,NaN,NaN,54.0,56.0
1,http://www.iedb.org/receptor/2,2,LA-2,heavylight,http://www.iedb.org/reference/280,http://www.iedb.org/epitope/190616,"S206, S207, A208, A209, T210, K211, K212, T213...",Outer surface protein A,Borreliella burgdorferi (Lyme disease spirochete),B cell,...,NaN,NaN,27.0,32.0,NaN,YTS,NaN,NaN,50.0,52.0
2,http://www.iedb.org/receptor/3,3,OKT3,heavylight,http://www.iedb.org/reference/310,http://www.iedb.org/epitope/2134650,"E56, G68, E70, R101, G102, S103, K104, P105, D107",T-cell surface glycoprotein CD3 epsilon chain,Homo sapiens (human),B cell,...,NaN,NaN,27.0,31.0,NaN,DTS,NaN,NaN,49.0,51.0
3,http://www.iedb.org/receptor/4,4,Fab-12 variant Y0317,heavylight,http://www.iedb.org/reference/312,http://www.iedb.org/epitope/2135423,"V: F43; W: Y71, K74, Q105, I106, M107, R108, I...",VEGF homodimer,Homo sapiens (human),B cell,...,NaN,NaN,27.0,32.0,NaN,FTS,NaN,NaN,50.0,52.0
4,http://www.iedb.org/receptor/4,4,Fab-12 variant Y0317,heavylight,http://www.iedb.org/reference/312,http://www.iedb.org/epitope/2135423,"V: F43; W: Y71, K74, Q105, I106, M107, R108, I...",VEGF homodimer,Homo sapiens (human),B cell,...,NaN,NaN,27.0,32.0,NaN,FTS,NaN,NaN,50.0,52.0


In [3]:
# human BCRs
bcr = df.loc[df["Organism IRI"] == "http://purl.obolibrary.org/obo/NCBITaxon_9606"]
bcr.shape

(5005, 70)

In [4]:
bcr["Source Molecule"].value_counts()

Source Molecule
glycolipid                                                                777
surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]    541
Spike glycoprotein                                                        241
Circumsporozoite protein                                                  223
Envelope glycoprotein gp160                                               161
                                                                         ... 
complement decay-accelerating factor isoform X1 [Homo sapiens]              1
pregnancy zone protein [Homo sapiens]                                       1
lactoferrin [Homo sapiens]                                                  1
complement C4d                                                              1
Angiogenin                                                                  1
Name: count, Length: 759, dtype: int64

In [5]:
bcr["Source Organism"].value_counts()[:20]

Source Organism
Homo sapiens (human)                                                                                        892
Sus scrofa (pig)                                                                                            780
SARS-CoV2                                                                                                   552
Human immunodeficiency virus 1 (human immunodeficiency virus 1 HIV-1)                                       395
Plasmodium falciparum (malaria parasite P. falciparum)                                                      272
Zaire ebolavirus Makona (Ebola virus Makona)                                                                 92
Arachis hypogaea (peanut)                                                                                    86
Ebola virus - Mayinga, Zaire, 1976 (Ebola virus (strain Zaire Mayinga))                                      76
SARS-CoV2 Omicron                                                                       

In [6]:
# COVID vs HIV
bcr = bcr.loc[
    bcr["Source Organism"].isin(
        [
            "SARS-CoV2",
            "Human immunodeficiency virus 1 (human immunodeficiency virus 1 HIV-1)",
        ]
    )
]
bcr.shape

(947, 70)

In [7]:
# heavy chain sequence
bcr = bcr.dropna(subset="Protein Sequence").drop_duplicates(subset="Protein Sequence")
bcr.shape

(509, 70)

In [8]:
bcr["Source Organism"].value_counts()

Source Organism
SARS-CoV2                                                                313
Human immunodeficiency virus 1 (human immunodeficiency virus 1 HIV-1)    196
Name: count, dtype: int64

In [9]:
# subset columns
bcr = bcr.loc[:, ["Protein Sequence", "Source Organism"]]
bcr.columns = ["sequence", "target"]
bcr["target"] = bcr["target"].map(
    lambda x: "SARS-CoV2" if x == "SARS-CoV2" else "HIV-1"
)
bcr.reset_index(inplace=True, drop=True)
bcr.head()

,sequence,target
0,QVQLVQSGAEVKRPGSSVTVSCKASGGSFSTYALSWVRQAPGRGLE...,HIV-1
1,QVQLVQSGAEVKRPGSSVTVSCKASGGSFSTYALSWVRQAPGRGLE...,HIV-1
2,QVQLVQSGAEVKRPGSSVTVSCKASGGSFSTYALSWVRQAPGRGLE...,HIV-1
3,QVQLVQSGAEVKRPGSSVTVSCKASGGSFSTYALSWVRQAPGRGLE...,HIV-1
4,SEVQLLESGPGLLKPSETLSLTCTVSGGSMINYYWSWIRQPPGERP...,HIV-1


In [10]:
bcr["target"].value_counts()

target
SARS-CoV2    313
HIV-1        196
Name: count, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# split
train, test = train_test_split(
    bcr, stratify=bcr["target"], random_state=0, test_size=0.15
)

In [13]:
train.shape, test.shape

((432, 2), (77, 2))

In [14]:
bcr.to_parquet("../data/bcr.parquet")
train.to_parquet("../data/bcr_train.parquet")
test.to_parquet("../data/bcr_test.parquet")